In [1]:
# libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import dataset
data = pd.read_csv('data/Boston/result.csv')
print(data.shape)
data.head()

(3585, 68)


,price,neighbourhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,square_feet,...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,250.0,Roslindale,House,Entire home/apt,4,1.5,2.0,3.0,Real Bed,NaN,...,0,1,1,0,0,1,1,0,0,1
1,65.0,Roslindale,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,NaN,...,0,1,1,0,0,1,1,0,0,1
2,65.0,Roslindale,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,NaN,...,1,1,1,0,0,1,1,0,0,1
3,75.0,Roslindale,House,Private room,4,1.0,1.0,2.0,Real Bed,NaN,...,1,1,1,0,0,1,1,0,0,1
4,79.0,Roslindale,House,Private room,2,1.5,1.0,2.0,Real Bed,NaN,...,0,1,1,0,0,0,0,0,0,1


In [3]:
# create variable of interest price per accommodate (ppa)
data.insert(loc=0, column='ppa', value=data['price']/data['accommodates'])
# drop price and accommodate
data = data.drop(columns=['price', 'accommodates'])

### Null values

1. Delete square feet: more than 95% of null values.
2. Numerical variables: input null values with the median → there are extreme values, the mean does not seem to be a good measurement of the center in this case. 
3. Categorical variables: input the mode.

In [4]:
# delete square feet 
data = data.drop(columns=['square_feet'])

In [5]:
# show number of null values per column in numerical variables
numerical = data.select_dtypes(include=['float64', 'int64'])
numerical = numerical[numerical.columns[1:15]]
numerical.columns
data[numerical.columns].isnull().sum()

bathrooms                       14
bedrooms                        10
beds                             9
host_since                       0
host_response_rate             471
host_acceptance_rate           471
number_of_reviews                0
reviews_per_month              756
review_scores_accuracy         823
review_scores_cleanliness      818
review_scores_checkin          820
review_scores_communication    818
review_scores_location         822
review_scores_value            821
dtype: int64

In [6]:
# fill nulls with median in numerical variables
for col in numerical.columns:
    m = data[col].median()
    data[col] = data[col].fillna(m)

In [7]:
# verify that there are not nulls 
data[numerical.columns].isnull().sum()

bathrooms                      0
bedrooms                       0
beds                           0
host_since                     0
host_response_rate             0
host_acceptance_rate           0
number_of_reviews              0
reviews_per_month              0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
dtype: int64

In [8]:
# show the null values in categorical variables
cat = data.select_dtypes(exclude=['float64', 'int64'])
data[cat.columns].isnull().sum()

neighbourhood               0
property_type               3
room_type                   0
bed_type                    0
cancellation_policy         0
host_response_time        471
host_is_superhost           0
host_identity_verified      0
dtype: int64

In [9]:
# fill nulls with the mode in the categorical features
for col in cat.columns:
    mode = data[col].mode()[0]
    data[col] = data[col].fillna(mode)

In [10]:
# verify that there are no nulls
data[cat.columns].isnull().sum()

neighbourhood             0
property_type             0
room_type                 0
bed_type                  0
cancellation_policy       0
host_response_time        0
host_is_superhost         0
host_identity_verified    0
dtype: int64

In [11]:
# no nulls in the whole dataset
print('number of null values in the whole dataset:', data.isnull().sum().sum())

number of null values in the whole dataset: 0


In [12]:
data.head()

,ppa,neighbourhood,property_type,room_type,bathrooms,bedrooms,beds,bed_type,cancellation_policy,host_since,...,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,62.50,Roslindale,House,Entire home/apt,1.5,2.0,3.0,Real Bed,moderate,511,...,0,1,1,0,0,1,1,0,0,1
1,32.50,Roslindale,Apartment,Private room,1.0,1.0,1.0,Real Bed,moderate,1553,...,0,1,1,0,0,1,1,0,0,1
2,32.50,Roslindale,Apartment,Private room,1.0,1.0,1.0,Real Bed,moderate,2676,...,1,1,1,0,0,1,1,0,0,1
3,18.75,Roslindale,House,Private room,1.0,1.0,2.0,Real Bed,moderate,1235,...,1,1,1,0,0,1,1,0,0,1
4,39.50,Roslindale,House,Private room,1.5,1.0,2.0,Real Bed,flexible,850,...,0,1,1,0,0,0,0,0,0,1


### From categorical to dummy

In [13]:
# select the categorical variables and create dummy
categorical = data[cat.columns]
dummies = pd.get_dummies(categorical)
dummies.head()

,neighbourhood_Allston,neighbourhood_Back Bay,neighbourhood_Bay Village,neighbourhood_Beacon Hill,neighbourhood_Brighton,neighbourhood_Charlestown,neighbourhood_Chinatown,neighbourhood_Dorchester,neighbourhood_Downtown,neighbourhood_East Boston,...,cancellation_policy_strict,cancellation_policy_super_strict_30,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [14]:
# drop the categorical features from data
data = data.drop(columns=cat.columns)
# concatenate with dummies
data = pd.concat([data, dummies], axis=1)
data.head()

,ppa,bathrooms,bedrooms,beds,host_since,host_response_rate,host_acceptance_rate,number_of_reviews,reviews_per_month,review_scores_accuracy,...,cancellation_policy_strict,cancellation_policy_super_strict_30,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t
0,62.50,1.5,2.0,3.0,511,100.0,94.0,0,1.17,10.0,...,0,0,0,0,0,1,1,0,1,0
1,32.50,1.0,1.0,1.0,1553,100.0,100.0,36,1.30,10.0,...,0,0,0,0,0,1,1,0,0,1
2,32.50,1.0,1.0,1.0,2676,100.0,88.0,41,0.47,10.0,...,0,0,0,0,1,0,0,1,0,1
3,18.75,1.0,1.0,2.0,1235,100.0,50.0,1,1.00,10.0,...,0,0,0,0,1,0,1,0,1,0
4,39.50,1.5,1.0,2.0,850,100.0,100.0,29,2.25,10.0,...,0,0,0,0,0,1,0,1,0,1


In [15]:
data.describe()

,ppa,bathrooms,bedrooms,beds,host_since,host_response_rate,host_acceptance_rate,number_of_reviews,reviews_per_month,review_scores_accuracy,...,cancellation_policy_strict,cancellation_policy_super_strict_30,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t
count,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.00000,3585.000000,3585.000000,3585.000000,3585.000000,...,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000,3585.000000
mean,62.786246,1.220781,1.255230,1.607531,894.668340,95.64742,85.464156,19.044630,1.802014,9.562064,...,0.441283,0.023710,0.011994,0.130823,0.339749,0.517434,0.886471,0.113529,0.273919,0.726081
std,54.900654,0.500698,0.752129,1.010933,620.783376,11.78841,20.567531,35.571658,1.911809,0.852131,...,0.496610,0.152165,0.108875,0.337254,0.473690,0.499766,0.317282,0.317282,0.446030,0.446030
min,5.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.010000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37.500000,1.000000,1.000000,1.000000,443.000000,98.00000,79.000000,1.000000,0.640000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,51.666667,1.000000,1.000000,1.000000,765.000000,100.00000,94.000000,5.000000,1.170000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,75.000000,1.000000,2.000000,2.000000,1230.000000,100.00000,100.000000,21.000000,2.130000,10.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,1372.000000,6.000000,5.000000,16.000000,2857.000000,100.00000,100.000000,404.000000,19.150000,10.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
data.shape

(3585, 116)

### Detect and remove outliers with Isolation Forest

In [17]:
# import libraries
from sklearn.ensemble import IsolationForest

In [18]:
# create X matrix
Xcols = list(data.columns)[1:116]
X = data[Xcols]
X.shape

(3585, 115)

In [19]:
# create y vector
y = data['ppa']
y.shape

(3585,)

In [20]:
# define model
iforest = IsolationForest(n_jobs = -1, contamination=0.05, random_state=333)
# fit and predict
pred= iforest.fit_predict(X)

In [21]:
# add the scores and the labels to the original data to detect outliers
data['scores'] = iforest.decision_function(X)
data['anomaly_label'] = pred

In [22]:
# 5% of outliers detected
gb = pd.DataFrame((data.groupby(by='anomaly_label').count()['ppa']/data.shape[0]))*100
gb.columns = ['%']
round(gb,2)

,%
anomaly_label,
-1,5.02
1,94.98


In [23]:
# take a look into outliers
outliers = data[data.anomaly_label==-1]
outliers.shape

(180, 118)

In [24]:
outliers.head()

,ppa,bathrooms,bedrooms,beds,host_since,host_response_rate,host_acceptance_rate,number_of_reviews,reviews_per_month,review_scores_accuracy,...,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t,scores,anomaly_label
5,37.5,1.0,1.0,1.0,168,100.0,95.0,8,1.70,10.0,...,0,0,1,0,0,1,0,1,-0.006752,-1
7,37.5,2.0,1.0,1.0,1494,100.0,100.0,67,2.38,10.0,...,0,0,1,0,0,1,0,1,-0.007579,-1
8,29.0,1.0,1.0,2.0,1414,100.0,100.0,65,5.36,10.0,...,0,0,0,1,0,1,0,1,-0.021111,-1
11,57.0,1.0,1.0,1.0,1089,100.0,89.0,17,0.48,9.0,...,0,0,0,1,0,1,0,1,-0.005933,-1
25,47.5,1.5,1.0,1.0,1327,100.0,92.0,40,1.13,10.0,...,0,0,0,1,0,1,0,1,-0.005463,-1


In [25]:
round(outliers.describe(),2)

,ppa,bathrooms,bedrooms,beds,host_since,host_response_rate,host_acceptance_rate,number_of_reviews,reviews_per_month,review_scores_accuracy,...,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t,scores,anomaly_label
count,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.00,...,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.00,180.0
mean,65.20,1.29,1.27,1.66,887.67,92.33,82.72,21.73,1.75,9.02,...,0.04,0.19,0.33,0.44,0.77,0.23,0.39,0.61,-0.02,-1.0
std,93.77,0.75,0.87,1.39,706.23,19.72,22.97,38.94,1.92,1.61,...,0.21,0.39,0.47,0.50,0.42,0.42,0.49,0.49,0.02,0.0
min,8.57,0.00,0.00,0.00,15.00,0.00,0.00,0.00,0.03,2.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.08,-1.0
25%,34.31,1.00,1.00,1.00,331.25,96.75,67.75,1.00,0.48,9.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,-0.03,-1.0
50%,45.00,1.00,1.00,1.00,717.00,100.00,94.00,5.50,1.14,10.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,-0.01,-1.0
75%,65.50,1.50,1.00,2.00,1414.25,100.00,100.00,25.25,2.49,10.00,...,0.00,0.00,1.00,1.00,1.00,0.00,1.00,1.00,-0.01,-1.0
max,999.00,6.00,5.00,9.00,2760.00,100.00,100.00,271.00,11.39,10.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,-0.00,-1.0


In [26]:
# remove outliers
data = data[data.anomaly_label== 1]
# remove outlier labels and scores
data = data[data.columns[0:116]]

In [27]:
data.head()

,ppa,bathrooms,bedrooms,beds,host_since,host_response_rate,host_acceptance_rate,number_of_reviews,reviews_per_month,review_scores_accuracy,...,cancellation_policy_strict,cancellation_policy_super_strict_30,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_is_superhost_f,host_is_superhost_t,host_identity_verified_f,host_identity_verified_t
0,62.50,1.5,2.0,3.0,511,100.0,94.0,0,1.17,10.0,...,0,0,0,0,0,1,1,0,1,0
1,32.50,1.0,1.0,1.0,1553,100.0,100.0,36,1.30,10.0,...,0,0,0,0,0,1,1,0,0,1
2,32.50,1.0,1.0,1.0,2676,100.0,88.0,41,0.47,10.0,...,0,0,0,0,1,0,0,1,0,1
3,18.75,1.0,1.0,2.0,1235,100.0,50.0,1,1.00,10.0,...,0,0,0,0,1,0,1,0,1,0
4,39.50,1.5,1.0,2.0,850,100.0,100.0,29,2.25,10.0,...,0,0,0,0,0,1,0,1,0,1


In [28]:
data.shape

(3405, 116)

In [29]:
# save this dataset to answer the first question lately
data.to_csv('data/Boston/data_clean.csv', index=False)